# Quickstart: In-App Labeling in 10 Minutes

This quickstart shows you how to create and edit labels directly in the FiftyOne App. By the end, you'll know:

1. How to enter Annotate mode
2. How to draw bounding boxes and set classes
3. Where your labels are stored
4. How to export labeled data

Let's go.

> **Note:** This track is standalone. The Full Loop track uses a separate dataset, so you can do both independently.

## Setup

First, define the label field name we'll use throughout this tutorial. **Use this exact name** when creating your field in the App.

In [ ]:
# IMPORTANT: Use this exact field name when labeling in the App
LABEL_FIELD = "my_labels"
DATASET_NAME = "my_annotation_project"

print(f"Dataset name: {DATASET_NAME}")
print(f"Label field: {LABEL_FIELD}")
print(f"\nWhen you create a label field in the App, name it exactly: {LABEL_FIELD}")

In [ ]:
!pip install -U fiftyone

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F

## Load and Clone the Dataset

We'll use the `quickstart` dataset (200 images from COCO). We clone it so your annotations are saved separately from the original zoo dataset.

This cell is **idempotent** - safe to rerun. If the dataset already exists, it loads it.

In [ ]:
# Load or create the dataset (safe to rerun)
if DATASET_NAME in fo.list_datasets():
    print(f"Loading existing dataset: {DATASET_NAME}")
    dataset = fo.load_dataset(DATASET_NAME)
else:
    print(f"Creating new dataset: {DATASET_NAME}")
    source = foz.load_zoo_dataset("quickstart")
    dataset = source.clone(DATASET_NAME)
    dataset.persistent = True

print(f"\nDataset: {dataset.name}")
print(f"Samples: {len(dataset)}")
print(f"Persistent: {dataset.persistent}")

## Launch the App

In [ ]:
session = fo.launch_app(dataset)

## Enter Annotate Mode

In the FiftyOne App:

1. **Click on any sample** to open the sample modal (expanded view)
2. **Look at the left sidebar** - you'll see tabs at the top
3. **Click the "Annotate" tab** (pencil icon) to enter annotation mode

When you enter Annotate mode, you'll see:
- **Action buttons** at the top: Classification, Detection (bounding box), Undo, Redo
- **Schema button**: Configure which fields and classes are available
- **Label list**: Shows existing labels on this sample

## Create the Label Field

Before drawing boxes, create a field to store them:

1. In Annotate mode, click the **Schema** button
2. Click **"+ Add Field"**
3. **Enter the exact name:** `my_labels` (must match `LABEL_FIELD` above)
4. Select the field type: **Detections** for bounding boxes
5. Define your classes (e.g., `car`, `person`, `bicycle`)
6. Click **Save**

Your new field is now available for annotation.

## Draw Bounding Boxes

Now let's draw some boxes:

1. **Click the Detection button** (square icon) in the action bar
2. **Click and drag** on the image to draw a bounding box
3. **Release** to complete the box
4. **Select a class** from the dropdown that appears
5. Your label is saved automatically!

### Tips:
- **Resize**: Drag the corners or edges of a box
- **Move**: Click inside the box and drag
- **Delete**: Select the box and press `Delete` or `Backspace`
- **Undo/Redo**: Use the buttons or keyboard shortcuts

### Try it now:
1. Go back to the App
2. Draw 3-5 bounding boxes on different objects
3. Assign classes to each
4. Move to a few more samples and label those too

## Verify Your Labels Saved

**Run this cell after labeling to confirm your annotations were saved correctly.**

If the counts below are 0, either:
- You haven't labeled any samples yet, or
- You used a different field name (must be exactly `my_labels`)

In [ ]:
# Reload to see your changes
dataset.reload()

# Count samples with labels in your field
if LABEL_FIELD in dataset.get_field_schema():
    labeled = dataset.match(F(f"{LABEL_FIELD}.detections").length() > 0)
    total_dets = sum(
        len(s[LABEL_FIELD].detections) 
        for s in labeled 
        if s[LABEL_FIELD] is not None
    )
    
    print(f"Samples with labels in '{LABEL_FIELD}': {len(labeled)}")
    print(f"Total detections: {total_dets}")
    
    if len(labeled) > 0:
        print(f"\n--- Example from first labeled sample ---")
        sample = labeled.first()
        print(f"File: {sample.filepath.split('/')[-1]}")
        for det in sample[LABEL_FIELD].detections:
            print(f"  {det.label}: {[round(x, 3) for x in det.bounding_box]}")
    else:
        print("\n>>> No labels found. Did you draw boxes in the App?")
        print(f">>> Make sure your field is named exactly: {LABEL_FIELD}")
else:
    print(f"Field '{LABEL_FIELD}' not found in dataset.")
    print(f"\nAvailable fields: {list(dataset.get_field_schema().keys())}")
    print(f"\n>>> Create a field named exactly '{LABEL_FIELD}' in the App.")

## Create Classifications (Optional)

You can also add image-level classifications:

1. **Click the Classification button** (tag icon)
2. **Select a class** from the dropdown
3. The classification is added to the sample

Use classifications for:
- Image-level attributes ("indoor", "outdoor", "daytime", "night")
- Quality flags ("blurry", "good", "needs_review")
- Dataset splits ("train", "val", "test")

## Edit Existing Labels

To edit labels that already exist on a sample:

1. **Click on a label** in the image or in the sidebar list
2. The label becomes selected (highlighted)
3. **Drag** to reposition or resize
4. **Click the class name** to change it
5. **Press Delete** to remove it

Changes are saved automatically—no "Save" button needed.

## Export Your Labeled Data

Once you've annotated samples, export them for training. We filter to only samples that have labels.

In [ ]:
# Get only samples with labels
if LABEL_FIELD in dataset.get_field_schema():
    labeled_samples = dataset.match(F(f"{LABEL_FIELD}.detections").length() > 0)
else:
    labeled_samples = dataset.limit(0)

if len(labeled_samples) == 0:
    print("No labeled samples to export. Label some samples first!")
else:
    # Export in COCO format
    labeled_samples.export(
        export_dir="/tmp/my_labeled_data",
        dataset_type=fo.types.COCODetectionDataset,
        label_field=LABEL_FIELD,
    )
    print(f"Exported {len(labeled_samples)} samples to /tmp/my_labeled_data")

In [ ]:
# Or export in YOLO format for YOLOv8 training
if len(labeled_samples) > 0:
    labeled_samples.export(
        export_dir="/tmp/my_labeled_data_yolo",
        dataset_type=fo.types.YOLOv5Dataset,
        label_field=LABEL_FIELD,
    )
    print(f"Exported {len(labeled_samples)} samples to /tmp/my_labeled_data_yolo")

## Key Behaviors

### Auto-save
Labels save automatically as you create/edit them. There's no "Save" button.

### Schema Enforcement
If you define a schema with specific classes, only those classes are available when labeling. This prevents typos and maintains consistency.

### Persistence
Because we set `dataset.persistent = True`, your dataset and labels persist between Python sessions. Run `fo.load_dataset("my_annotation_project")` to reload it later.

## Summary

You learned how to:

1. Create a persistent dataset for annotation
2. Enter Annotate mode (click sample -> Annotate tab)
3. Create a label field with a schema
4. Draw bounding boxes and assign classes
5. Verify labels saved correctly
6. Export for training

**That's in-app annotation in FiftyOne.** You now have everything you need to label small datasets or make quick corrections.

---

## What's Next?

For production annotation workflows, continue to the **Full Loop Track**:

- **Step 2: Setup Splits** - Create proper train/val/test splits
- **Step 3: Smart Selection** - Use diversity sampling to pick high-value samples
- **Step 4: Annotation + QA** - Disciplined labeling with quality checks
- **Step 5: Train + Evaluate** - Train a model and analyze failures
- **Step 6: Iteration** - Use failures to drive the next labeling batch

The Full Loop teaches you to label **smarter, not harder**.

> **Note:** The Full Loop uses a separate dataset (`annotation_tutorial`) so you can do both tracks independently.